# Coronavirus (COVID-19) England Cases Scraper
## David Beavan @DavidBeavan
## Licence: MIT. Sources: see below
## Notes
* This is liable to break, lots if the official sources change (and they do)

In [1]:
# load libraries
from pathlib import Path
import pandas as pd
import requests
import datetime
import shutil
import json

In [2]:
# Prep data
# Data is not packaged with code, Will download from sources and save for future runs
data_base_dir = Path('data/secondary_sources')
data_base_dir.mkdir(parents=True, exist_ok=True)

data_sub_dir = data_base_dir.joinpath('utla_cases_table')
data_sub_dir.mkdir(parents=True, exist_ok=True)

utla_cases_file = data_sub_dir.joinpath('utla_cases.csv')

In [3]:
# Fetch most recent cases data

# Data is in a csv file, but under a permanent redirect
# Therefore, we need to use requests first
r = requests.head(
    'https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv',
    allow_redirects=True)
utla_cases_dashboard_df = pd.read_csv(r.url)

# Filter to just utlas
utla_cases_dashboard_df = utla_cases_dashboard_df.loc[
    utla_cases_dashboard_df['Area type'] == 'Upper tier local authority']

# Clean up unneeded columns
utla_cases_dashboard_df.drop(columns=['Area type', 'Daily lab-confirmed cases', 'Previously reported daily cases', 'Change in daily cases',
                                      'Previously reported cumulative cases', 'Change in cumulative cases', 'Cumulative lab-confirmed cases rate'])

# Create lookup table for area code <-> area name
# Use this because the pivot later only takes one index, and this is
# effectivly a compound index
area_code_to_name = utla_cases_dashboard_df[[
    'Area code', 'Area name']].drop_duplicates()
area_code_to_name

# Un-tidy the data for ease of use
utla_cases_dashboard_df = utla_cases_dashboard_df.pivot(
    index='Area code',
    columns='Specimen date',
    values='Cumulative lab-confirmed cases')

# Place area name back in the dataframe
utla_cases_df = pd.merge(area_code_to_name, utla_cases_dashboard_df,
                         left_on='Area code',
                         right_on='Area code',
                         how='left')

# Rename columns to their old names, from old data sources
utla_cases_df.rename(
    columns={
        'Area code': 'utla',
        'Area name': 'GSS_NM'},
    inplace=True)

# Re-index and sort
utla_cases_df.set_index('utla', inplace=True)
utla_cases_df.sort_values(['utla'], inplace=True)

In [4]:
# Preview data
# utla_cases_df

In [5]:
# Put date columns in chronological order
cols = utla_cases_df.columns.tolist()

non_date_cols = cols[:1]
date_cols = cols[1:]

utla_cases_non_date_cols_df = utla_cases_df[non_date_cols]

date_cols.sort()
cols = non_date_cols + date_cols
utla_cases_date_cols_df = utla_cases_df[date_cols]


# Insert new columns even if no case reports were mad eon tthat day (e.g.
# early on in the pandemic)
min_date = min(date_cols)
max_date = max(date_cols)
date_range = pd.date_range(min_date, max_date).strftime('%Y-%m-%d')
utla_cases_date_cols_df = utla_cases_date_cols_df.reindex(columns=date_range)

# Forward fill cases, duplicating case numbers if no new reports were made to made a full, non-sparse dataframe
utla_cases_df = pd.concat(
    [utla_cases_non_date_cols_df, utla_cases_date_cols_df.ffill(axis=1)], axis=1)

# Start all cases at zero, unless otherwise given
utla_cases_df.fillna(0, inplace=True)

# Set cases back to int from float, as int could not have handeled the NaNs we just removed
for col in date_range:
    utla_cases_df = utla_cases_df.astype({col: int})

In [6]:
# Preview data
utla_cases_df.head()

,GSS_NM,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,...,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20
utla,,,,,,,,,,,,,,,,,,,,,
E06000001,Hartlepool,0,0,0,0,0,0,0,0,0,...,278,289,295,302,306,310,310,319,319,319
E06000002,Middlesbrough,0,0,0,0,0,0,0,0,0,...,633,634,635,635,636,638,642,651,652,652
E06000003,Redcar and Cleveland,0,0,0,0,0,0,0,0,0,...,376,379,384,385,385,387,389,395,399,399
E06000004,Stockton-on-Tees,0,0,0,0,0,0,0,0,0,...,485,495,502,508,517,524,526,539,548,548
E06000005,Darlington,0,0,0,0,0,0,0,0,0,...,343,349,350,351,352,353,354,358,358,358


In [7]:
# Preview data
utla_cases_df.sum()

GSS_NM        HartlepoolMiddlesbroughRedcar and ClevelandSto...
2020-01-30                                                    1
2020-01-31                                                    2
2020-02-01                                                    2
2020-02-02                                                    2
                                    ...                        
2020-05-16                                               138884
2020-05-17                                               139458
2020-05-18                                               140180
2020-05-19                                               140507
2020-05-20                                               140540
Length: 113, dtype: object

In [8]:
# Save data
utla_cases_df.to_csv(utla_cases_file)

# Delete old backup
backup_dir = Path('data/secondary_sources_bak/secondary_sources_bak_' +
                  datetime.date.today().isoformat())
if backup_dir.exists():
    shutil.rmtree(backup_dir)

# Make new backup
shutil.copytree(Path('data/secondary_sources'), backup_dir)

PosixPath('data/secondary_sources_bak/secondary_sources_bak_2020-05-21')